In [84]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold

from pycaret.regression import *
import lightgbm as lgb
import xgboost
import optuna.integration.lightgbm as lgb_opt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

pd.set_option('display.max_columns', 30)

In [28]:
train_full = pd.read_csv('../input/train.csv')

In [29]:
test = pd.read_csv('../input/test.csv')

In [30]:
print(train_full.shape)
train_full.head()

(300000, 26)


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [31]:
train.isnull().sum()

id        0
cat0      0
cat1      0
cat2      0
cat3      0
cat4      0
cat5      0
cat6      0
cat7      0
cat8      0
cat9      0
cont0     0
cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
cont11    0
cont12    0
cont13    0
dtype: int64

In [32]:
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [33]:
train = train_full.drop(['target'], axis=1)
y = train_full['target']

In [34]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

In [35]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test[my_cols].copy()

In [36]:
# Preprocessing for numerical data
# 数値データ欠損値の補完 ⇒　欠損値なしなのでイラン
#numerical_transformer = SimpleImputer(strategy='constant')

In [79]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        # ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
# model = RandomForestRegressor(n_estimators=100, random_state=0)
model =  lgb.LGBMRegressor(random_state=420)
# model =  xgboost.XGBRegressor(random_state=0)


# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('RMSE:', mean_squared_error(y_valid, preds)**0.5)

RMSE: 0.7463135177805479


In [80]:
d = {'id' : X_test['id'], 'target' : clf.predict(X_test)}
preds_test = pd.DataFrame.from_dict(d)

In [82]:
preds_test.to_csv('../output/submission_lgbm.csv', index=False)

In [81]:
preds_test

,id,target
0,0,8.139080
1,5,8.251010
2,15,8.323052
3,16,8.185089
4,17,8.251100
...,...,...
199995,499987,8.233882
199996,499990,8.314312
199997,499991,8.214550
199998,499994,8.238098


In [85]:
df_train_set = train_full.iloc[:int(len(train_full)*0.8),:]
df_valid_set = train_full.iloc[int(len(train_full)*0.8):,:]

cv = KFold(shuffle=True)

exp1 = setup(data = df_train_set, 
             test_data = df_valid_set,
             target = 'target', 
             # ignore_features = None, 
             fold_shuffle= True, 
             fold_strategy = cv ,
             data_split_shuffle = True)   

,Description,Value
0,session_id,4557
1,Target,target
2,Original Data,"(240000, 26)"
3,Missing Values,False
4,Numeric Features,15
5,Categorical Features,10
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(240000, 68)"


In [86]:
best = compare_models(['lightgbm', 'xgboost', 'catboost'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.5738,0.5214,0.7220,0.0653,0.0788,0.0707,32.8380
lightgbm,Light Gradient Boosting Machine,0.5758,0.5262,0.7254,0.0565,0.0792,0.0710,3.7580
xgboost,Extreme Gradient Boosting,0.5771,0.5276,0.7264,0.0541,0.0792,0.0711,39.5520


In [95]:
preds_test = predict_model(best, data=test)['Label']


In [96]:
d = {'id' : test['id'], 'target' : preds_test}
preds_test = pd.DataFrame.from_dict(d)

In [97]:
preds_test.to_csv('../output/submission_pycaret.csv', index=False)